# **Durée de retour à l'emploi : phase exploratoire**

In [1]:
# nécessité d'importer la librairie xlrd pour permettre à Python de lire Excel (les données INSEE ne sont pas dispobinle en csv)
!pip install xlrd

In [2]:
!pip install openpyxl
# également demandé par Python pour lire Excel

In [3]:
import pandas as pd
import xlrd
import openpyxl
import numpy as np

## 1. Constitution de la BDD OPEN DATA

4 fichiers sont nécessaires à la constitution de cette df :
- 2 indicateurs de dynamisme du marché du travail : taux de chômage et indice de concentration de l'emploi
- 2 codes pour opérer les jointures : code INSEE de la commune (jointure avec les codes postaux) et de la Zone d'emploi (jointure avec les codes communes et le taux de chômage)

**Chargement du fichier de correspondance Code INSEE - Code postal**
- Source : La Poste sur Open Data

> *La base officielle des codes postaux est un jeu de données qui fournit la correspondance entre les codes postaux et les codes INSEE des communes, de France (métropole et DOM), des TOM, ainsi que de MONACO. Le fichier CSV a été mis à jour le 21 mars 2022*

- URL : https://www.data.gouv.fr/fr/datasets/base-officielle-des-codes-postaux/#resources

In [4]:
communes_df = pd.read_excel(r"C:\Users\emman\Documents\4_Diplômes\2022_DESU-AMU\0_Projet\projet_desu_tre\data\external\laposte_hexasmal_4.xlsx")
communes_df.head(10)

,code_commune_insee,nom_de_la_commune,code_postal,ligne_5,libelle_d_acheminement,coordonnees_gps
0,01001,L ABERGEMENT CLEMENCIAT,1400,NaN,L ABERGEMENT CLEMENCIAT,"46.153721024,4.925850148"
1,01002,L ABERGEMENT DE VAREY,1640,NaN,L ABERGEMENT DE VAREY,"46.009605679,5.428087796"
2,01004,AMBERIEU EN BUGEY,1500,NaN,AMBERIEU EN BUGEY,"45.961048852,5.372275427"
3,01005,AMBERIEUX EN DOMBES,1330,NaN,AMBERIEUX EN DOMBES,"45.99616357,4.911967138"
4,01006,AMBLEON,1300,NaN,AMBLEON,"45.749886304,5.594584599"
5,01007,AMBRONAY,1500,NaN,AMBRONAY,"46.005690539,5.357748739"
6,01008,AMBUTRIX,1500,NaN,AMBUTRIX,"45.936683389,5.332446671"
7,01009,ANDERT ET CONDON,1300,NaN,ANDERT ET CONDON,"45.787194157,5.657799976"
8,01010,ANGLEFORT,1350,NaN,ANGLEFORT,"45.909091467,5.795056945"
9,01011,APREMONT,1100,NaN,APREMONT,"46.205877513,5.657980818"


In [5]:
communes_df.shape

(38926, 6)

In [6]:
# modification et préparation de la table code_communes_df : suppression colonnes inutiles, en-têtes renommées
communes_df = communes_df.rename(columns = {'code_postal' : 'cp', 'code_commune_insee':'code_commune', 'nom_de_la_commune' : 'commune'})
communes_df = communes_df[['cp', 'code_commune', 'commune']]
communes_df

,cp,code_commune,commune
0,1400,01001,L ABERGEMENT CLEMENCIAT
1,1640,01002,L ABERGEMENT DE VAREY
2,1500,01004,AMBERIEU EN BUGEY
3,1330,01005,AMBERIEUX EN DOMBES
4,1300,01006,AMBLEON
...,...,...,...
38921,20272,2B356,ZALANA
38922,20214,2B361,ZILIA
38923,20272,2B364,ZUANI
38924,20243,2B365,SAN GAVINO DI FIUMORBO


In [7]:
#vérifier le nombre de valeurs manquantes dans chaque colonne
communes_df.isna().sum()

cp              0
code_commune    0
commune         0
dtype: int64

In [8]:
# conversion des données mixtes de la colonne "code commune" en string afin de pouvoir faire le merge
communes_df['code_commune'] = communes_df['code_commune'].astype(str)

In [9]:
#vérifier les types des colonnes : 
communes_df.dtypes

cp               int64
code_commune    object
commune         object
dtype: object

**Chargement du fichier base des zones d'emploi Code INSEE - ZE depuis INSEE**
- Source : INSEE - composition communale des zones d'emploi (au format XLSX)
> *Base des zones d'emploi 2020 au 1ᵉʳ janvier 2022. Permet de rattacher chaque commune à sa zone d'emploi, car le taux de chômage actualisé est disponible par zone d'emploi et non par commune*
- URL : https://www.insee.fr/fr/information/4652957

In [10]:
ze_df = pd.read_excel(r"C:\Users\emman\Documents\4_Diplômes\2022_DESU-AMU\0_Projet\projet_desu_tre\data\external\ZE2020_au_01-01-2022.xlsx", skiprows = 4, header = 1, sheet_name = 'Composition_communale')
ze_df

,CODGEO,LIBGEO,ZE2020,LIBZE2020,ZE2020_PARTIE_REG,DEP,REG
0,01001,L'Abergement-Clémenciat,8405,Bourg en Bresse,NaN,01,84
1,01002,L'Abergement-de-Varey,8405,Bourg en Bresse,NaN,01,84
2,01004,Ambérieu-en-Bugey,8405,Bourg en Bresse,NaN,01,84
3,01005,Ambérieux-en-Dombes,8434,Villefranche-sur-Saône,NaN,01,84
4,01006,Ambléon,8404,Belley,NaN,01,84
...,...,...,...,...,...,...,...
34950,97613,M'Tsangamouji,601,Mayotte,NaN,976,6
34951,97614,Ouangani,601,Mayotte,NaN,976,6
34952,97615,Pamandzi,601,Mayotte,NaN,976,6
34953,97616,Sada,601,Mayotte,NaN,976,6


In [11]:
# modification et préparation de la table code_ze_df : suppression colonnes inutiles, en-têtes renommées
ze_df = ze_df.rename(columns = {"CODGEO":'code_commune2', 
                                          "LIBGEO" : 'commune2',  
                                          "ZE2020" : 'code_ze',
                                         "LIBZE2020" : 'ze'})
ze_df = ze_df[["code_commune2", "commune2", "code_ze", "ze"]]
#Fixer le code_ze en format STRING
ze_df.code_ze = ze_df.code_ze.astype(str)
ze_df

,code_commune2,commune2,code_ze,ze
0,01001,L'Abergement-Clémenciat,8405,Bourg en Bresse
1,01002,L'Abergement-de-Varey,8405,Bourg en Bresse
2,01004,Ambérieu-en-Bugey,8405,Bourg en Bresse
3,01005,Ambérieux-en-Dombes,8434,Villefranche-sur-Saône
4,01006,Ambléon,8404,Belley
...,...,...,...,...
34950,97613,M'Tsangamouji,601,Mayotte
34951,97614,Ouangani,601,Mayotte
34952,97615,Pamandzi,601,Mayotte
34953,97616,Sada,601,Mayotte


In [12]:
#vérifier le nombre de valeurs manquantes dans chaque colonne
ze_df.isna().sum()

code_commune2    0
commune2         0
code_ze          0
ze               0
dtype: int64

In [13]:
ze_df.dtypes

code_commune2    object
commune2         object
code_ze          object
ze               object
dtype: object

In [14]:
# conversion des données mixtes de la colonne "code commune" en string afin de pouvoir faire le merge
ze_df['code_commune2'] = ze_df['code_commune2'].astype(str)

In [15]:
#vérification du type des colonnes
ze_df.dtypes

code_commune2    object
commune2         object
code_ze          object
ze               object
dtype: object

**MERGE sur les 2 premiers JDD CP et codes communes et ZE**
> *Constitution d'un seul jeu avec toutes les informations CP, code commune et nom de la commune, code ZE et nom de la ZE. Dans un second temps, on enlevera les colonnes inutiles (gardées pour vérification).*

> *ATTENTION : le 1er JDD comprenait 39 193 lignes (codes postaux) : suppression depuis Excel des 267 codes postaux 98XXX (Collectivités d’outre-mer non dotées du statut de collectivité territoriale cad non habitées de façon permanente de types : terres australes et antartiques) + MONACO (99) + 5 codes 97XXX non répertoriés > il a donc été réduit à 38926 lignes

> Le second en a uniquement 34 955 (codes communes et codes ZE). Il faudra donc vérifier quels codes postaux n'ont pas de codes INSEE (par hypothèse les CP d'arrondissements à Paris, Marseille, Lyon)

**Choix du MERGE : on privilégie le JDD le plus complet : sur les communes**
> Le JDD sur les ZE avec 34955 lignes semble suffisant. Si on réalise le MERGE depuis le JDD avec 38926 lignes, on se retrouve avec 57 lignes avec des données manquantes : les arrondissements de Paris, Marseille et Lyon.

> A l'inverse, le MERGE sur le JDD des ZE avec 34 955 lignes n'a que 3 données manquantes, correspondant à l'équivalent des libéllée et codes de ZE pour Paris, Marseille et Lyon. Cela nous permettra de compléter la donnée que pour ces lignes.

> Les 2 autres JDD utilisés ensuite jointent sur la ZE (taux de chômage) et le code commune sans les arrondissements.

In [16]:
#MERGE des 2 dataframe à partir de la clé code_commune = code_commune2 avec JDD_ze à gauche (34 955) où il y en a moins que dans le JDD_commune (38 931)
INSEE_df = pd.merge(ze_df, communes_df, how = "left", left_on="code_commune2", right_on="code_commune")
print (INSEE_df) #vérifier le nombre de valeurs manquantes (null ou NA) dans chaque colonne : return TRUE where it detects a missing value
INSEE_df.isna().sum()

      code_commune2                 commune2 code_ze                      ze  \
0             01001  L'Abergement-Clémenciat    8405         Bourg en Bresse   
1             01002    L'Abergement-de-Varey    8405         Bourg en Bresse   
2             01004        Ambérieu-en-Bugey    8405         Bourg en Bresse   
3             01005      Ambérieux-en-Dombes    8434  Villefranche-sur-Saône   
4             01006                  Ambléon    8404                  Belley   
...             ...                      ...     ...                     ...   
38867         97614                 Ouangani     601                 Mayotte   
38868         97615                 Pamandzi     601                 Mayotte   
38869         97616                     Sada     601                 Mayotte   
38870         97617                 Tsingoni     601                 Mayotte   
38871         97617                 Tsingoni     601                 Mayotte   

            cp code_commune            

code_commune2    0
commune2         0
code_ze          0
ze               0
cp               3
code_commune     3
commune          3
dtype: int64

In [17]:
# identification et visualisation des 3 lignes avec des valeurs manquantes : il s'agit des communes avec arrondissement
INSEE_df[INSEE_df.code_commune.isna()]

,code_commune2,commune2,code_ze,ze,cp,code_commune,commune
4750,13055,Marseille,9312,Marseille,NaN,NaN,NaN
30158,69123,Lyon,8421,Lyon,NaN,NaN,NaN
32574,75056,Paris,1109,Paris,NaN,NaN,NaN


In [18]:
# Affiche le nombre de valeurs présentes dans chaque colonne
INSEE_df.count()

code_commune2    38872
commune2         38872
code_ze          38872
ze               38872
cp               38869
code_commune     38869
commune          38869
dtype: int64

In [19]:
#MERGE inversé des 2 dataframeq à partir de la clé code_commune2 = code_commune avec JDD_commune (38 931) à gauche car il en a plus que JDD_ze (34 955) 
INSEE1_df = pd.merge(communes_df, ze_df, how = "left", left_on="code_commune", right_on="code_commune2")
print (INSEE1_df) #vérifier le nombre de valeurs manquantes (null ou NA) dans chaque colonne : return TRUE where it detects a missing value
INSEE1_df.isna().sum()

          cp code_commune                  commune code_commune2  \
0       1400        01001  L ABERGEMENT CLEMENCIAT         01001   
1       1640        01002    L ABERGEMENT DE VAREY         01002   
2       1500        01004        AMBERIEU EN BUGEY         01004   
3       1330        01005      AMBERIEUX EN DOMBES         01005   
4       1300        01006                  AMBLEON         01006   
...      ...          ...                      ...           ...   
38921  20272        2B356                   ZALANA         2B356   
38922  20214        2B361                    ZILIA         2B361   
38923  20272        2B364                    ZUANI         2B364   
38924  20243        2B365   SAN GAVINO DI FIUMORBO         2B365   
38925  20240        2B366                    CHISA         2B366   

                      commune2 code_ze                      ze  
0      L'Abergement-Clémenciat    8405         Bourg en Bresse  
1        L'Abergement-de-Varey    8405         Bourg 

cp                0
code_commune      0
commune           0
code_commune2    57
commune2         57
code_ze          57
ze               57
dtype: int64

**Mise à jour du JDD INSEE1 : complétion des valeurs manquantes**

Il s'agira d'abord d'identifier les valeurs manquantes du JDD INSEE1 à l'aide du JDD INSEE, puis de remplacer les NaN par les valeurs adéquates

In [20]:
# identification et visualisation des 57 lignes avec des valeurs manquantes : il s'agit des arrondissements et de certains codes 97
INSEE1_df[INSEE1_df.code_commune2.isna()]

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze
4832,13001,13201,MARSEILLE 01,NaN,NaN,NaN,NaN
4833,13002,13202,MARSEILLE 02,NaN,NaN,NaN,NaN
4834,13003,13203,MARSEILLE 03,NaN,NaN,NaN,NaN
4835,13004,13204,MARSEILLE 04,NaN,NaN,NaN,NaN
4836,13005,13205,MARSEILLE 05,NaN,NaN,NaN,NaN
4837,13006,13206,MARSEILLE 06,NaN,NaN,NaN,NaN
4838,13007,13207,MARSEILLE 07,NaN,NaN,NaN,NaN
4839,13008,13208,MARSEILLE 08,NaN,NaN,NaN,NaN
4840,13008,13208,MARSEILLE 08,NaN,NaN,NaN,NaN
4841,13008,13208,MARSEILLE 08,NaN,NaN,NaN,NaN


In [21]:
# identification et visualisation des 3 valeurs de destination : codes commune2 + nom2 + code ZE + ze pgénériques pour Marseille, Lyon et Paris
INSEE_df[INSEE_df.code_commune.isna()]

,code_commune2,commune2,code_ze,ze,cp,code_commune,commune
4750,13055,Marseille,9312,Marseille,NaN,NaN,NaN
30158,69123,Lyon,8421,Lyon,NaN,NaN,NaN
32574,75056,Paris,1109,Paris,NaN,NaN,NaN


In [22]:
#vérification de l'index des lignes Marseille > ATTENTION borne extérieure exclue
INSEE1_df[4832:4858]

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze
4832,13001,13201,MARSEILLE 01,NaN,NaN,NaN,NaN
4833,13002,13202,MARSEILLE 02,NaN,NaN,NaN,NaN
4834,13003,13203,MARSEILLE 03,NaN,NaN,NaN,NaN
4835,13004,13204,MARSEILLE 04,NaN,NaN,NaN,NaN
4836,13005,13205,MARSEILLE 05,NaN,NaN,NaN,NaN
4837,13006,13206,MARSEILLE 06,NaN,NaN,NaN,NaN
4838,13007,13207,MARSEILLE 07,NaN,NaN,NaN,NaN
4839,13008,13208,MARSEILLE 08,NaN,NaN,NaN,NaN
4840,13008,13208,MARSEILLE 08,NaN,NaN,NaN,NaN
4841,13008,13208,MARSEILLE 08,NaN,NaN,NaN,NaN


In [23]:
#Traitement des valeurs manquantes : sur le JDD INSEE1 > affecter le code commune et ze aux arrondissements de Marseille au lieu des valeurs NaN
INSEE1_df.loc[4832:4858,"code_commune2"] = "13055" #affecte le code commune 2
INSEE1_df.loc[4832:4858,"commune2"] = "Marseille" #affecte le commune 2
INSEE1_df.loc[4832:4858,"code_ze"] = "9312" #affecte le code ze
INSEE1_df.loc[4832:4858,"ze"] = "Marseille" #affecte le nom de la ze
INSEE1_df[4832:4858] #affiche les lignes avec les nouvelles valeurs

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze
4832,13001,13201,MARSEILLE 01,13055,Marseille,9312,Marseille
4833,13002,13202,MARSEILLE 02,13055,Marseille,9312,Marseille
4834,13003,13203,MARSEILLE 03,13055,Marseille,9312,Marseille
4835,13004,13204,MARSEILLE 04,13055,Marseille,9312,Marseille
4836,13005,13205,MARSEILLE 05,13055,Marseille,9312,Marseille
4837,13006,13206,MARSEILLE 06,13055,Marseille,9312,Marseille
4838,13007,13207,MARSEILLE 07,13055,Marseille,9312,Marseille
4839,13008,13208,MARSEILLE 08,13055,Marseille,9312,Marseille
4840,13008,13208,MARSEILLE 08,13055,Marseille,9312,Marseille
4841,13008,13208,MARSEILLE 08,13055,Marseille,9312,Marseille


In [24]:
#vérification de l'index des lignes Lyon > ATTENTION borne extérieure exclue
INSEE1_df[29964:29973]

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze
29964,69001,69381,LYON 01,NaN,NaN,NaN,NaN
29965,69002,69382,LYON 02,NaN,NaN,NaN,NaN
29966,69003,69383,LYON 03,NaN,NaN,NaN,NaN
29967,69004,69384,LYON 04,NaN,NaN,NaN,NaN
29968,69005,69385,LYON 05,NaN,NaN,NaN,NaN
29969,69006,69386,LYON 06,NaN,NaN,NaN,NaN
29970,69007,69387,LYON 07,NaN,NaN,NaN,NaN
29971,69008,69388,LYON 08,NaN,NaN,NaN,NaN
29972,69009,69389,LYON 09,NaN,NaN,NaN,NaN


In [25]:
#Traitement des valeurs manquantes : sur le JDD INSEE1 > affecter le code commune et ze aux arrondissements de Lyon au lieu des valeurs NaN
INSEE1_df.loc[29964:29973,"code_commune2"] = "69123" #affecte le code commune 2
INSEE1_df.loc[29964:29973,"commune2"] = "Lyon" #affecte le commune 2
INSEE1_df.loc[29964:29973,"code_ze"] = "8421" #affecte le code ze
INSEE1_df.loc[29964:29973,"ze"] = "Lyon" #affecte le nom de la ze
INSEE1_df[29964:29973] #affiche les lignes avec les nouvelles valeurs

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze
29964,69001,69381,LYON 01,69123,Lyon,8421,Lyon
29965,69002,69382,LYON 02,69123,Lyon,8421,Lyon
29966,69003,69383,LYON 03,69123,Lyon,8421,Lyon
29967,69004,69384,LYON 04,69123,Lyon,8421,Lyon
29968,69005,69385,LYON 05,69123,Lyon,8421,Lyon
29969,69006,69386,LYON 06,69123,Lyon,8421,Lyon
29970,69007,69387,LYON 07,69123,Lyon,8421,Lyon
29971,69008,69388,LYON 08,69123,Lyon,8421,Lyon
29972,69009,69389,LYON 09,69123,Lyon,8421,Lyon


In [26]:
#vérification de l'index des lignes Paris > ATTENTION borne extérieure exclue
INSEE1_df[32189:32210]

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze
32189,75001,75101,PARIS 01,NaN,NaN,NaN,NaN
32190,75002,75102,PARIS 02,NaN,NaN,NaN,NaN
32191,75003,75103,PARIS 03,NaN,NaN,NaN,NaN
32192,75004,75104,PARIS 04,NaN,NaN,NaN,NaN
32193,75005,75105,PARIS 05,NaN,NaN,NaN,NaN
32194,75006,75106,PARIS 06,NaN,NaN,NaN,NaN
32195,75007,75107,PARIS 07,NaN,NaN,NaN,NaN
32196,75008,75108,PARIS 08,NaN,NaN,NaN,NaN
32197,75009,75109,PARIS 09,NaN,NaN,NaN,NaN
32198,75010,75110,PARIS 10,NaN,NaN,NaN,NaN


In [27]:
#Traitement des valeurs manquantes : sur le JDD INSEE1 > affecter le code commune et ze aux arrondissements de Paris au lieu des valeurs NaN
INSEE1_df.loc[32189:32210,"code_commune2"] = "75056" #affecte le code commune 2
INSEE1_df.loc[32189:32210,"commune2"] = "Paris" #affecte le commune 2
INSEE1_df.loc[32189:32210,"code_ze"] = "1109" #affecte le code ze
INSEE1_df.loc[32189:32210,"ze"] = "Paris" #affecte le nom de la ze
INSEE1_df[32189:32210] #affiche les lignes avec les nouvelles valeurs

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze
32189,75001,75101,PARIS 01,75056,Paris,1109,Paris
32190,75002,75102,PARIS 02,75056,Paris,1109,Paris
32191,75003,75103,PARIS 03,75056,Paris,1109,Paris
32192,75004,75104,PARIS 04,75056,Paris,1109,Paris
32193,75005,75105,PARIS 05,75056,Paris,1109,Paris
32194,75006,75106,PARIS 06,75056,Paris,1109,Paris
32195,75007,75107,PARIS 07,75056,Paris,1109,Paris
32196,75008,75108,PARIS 08,75056,Paris,1109,Paris
32197,75009,75109,PARIS 09,75056,Paris,1109,Paris
32198,75010,75110,PARIS 10,75056,Paris,1109,Paris


In [28]:
#vérifier que INSEE1_df a toujours le même nombre de lignes (38926)
INSEE1_df
#puis ²afficher le nombre de valeurs manquantes
INSEE1_df.isna().sum()

cp               0
code_commune     0
commune          0
code_commune2    0
commune2         0
code_ze          0
ze               0
dtype: int64

In [29]:
INSEE1_df.dtypes

cp                int64
code_commune     object
commune          object
code_commune2    object
commune2         object
code_ze          object
ze               object
dtype: object

In [30]:
INSEE1_df

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze
0,1400,01001,L ABERGEMENT CLEMENCIAT,01001,L'Abergement-Clémenciat,8405,Bourg en Bresse
1,1640,01002,L ABERGEMENT DE VAREY,01002,L'Abergement-de-Varey,8405,Bourg en Bresse
2,1500,01004,AMBERIEU EN BUGEY,01004,Ambérieu-en-Bugey,8405,Bourg en Bresse
3,1330,01005,AMBERIEUX EN DOMBES,01005,Ambérieux-en-Dombes,8434,Villefranche-sur-Saône
4,1300,01006,AMBLEON,01006,Ambléon,8404,Belley
...,...,...,...,...,...,...,...
38921,20272,2B356,ZALANA,2B356,Zalana,9405,Ghisonaccia
38922,20214,2B361,ZILIA,2B361,Zilia,9403,Calvi
38923,20272,2B364,ZUANI,2B364,Zuani,9404,Corte
38924,20243,2B365,SAN GAVINO DI FIUMORBO,2B365,San-Gavino-di-Fiumorbo,9405,Ghisonaccia


**Ajout du taux de chômage au JDD INSEE1 (constitution du JDD INSEE2)**

- Source : INSEE.fr

Dans un 1er temps, import du JDD de l'INSEE avec les taux de chômage actualisé au 4ème trimeste 2021

> *Le JDD en format Excel comprend les taux de chômage localisés par zone d'emploi, en moyenne trimestrielle, en France hors Mayotte et Guyane.*

- URL : https://www.insee.fr/fr/statistiques/1893230

Le fichier contient 5 lignes de métadonnées à supprimer et seules les colonnes A (ZE2020 = numéro de la ZE), B (LIBZE2020 = nom de la ZE) et CB (2021-T4 = taux de chômage au 4ème trimestre 2021) sont intéressantes à garder

In [31]:
# import du fichier avec suppression des métadonnées
chomage_df = pd.read_excel(r"C:\Users\emman\Documents\4_Diplômes\2022_DESU-AMU\0_Projet\projet_desu_tre\data\external\chomage-zone-t1-2003-t4-2021.xlsx",  skiprows = 4, header = 1, sheet_name = 'txcho_ze')
chomage_df

,ZE2020,LIBZE2020,REG,LIBREG,2003-T1,2003-T2,2003-T3,2003-T4,2004-T1,2004-T2,...,2019-T3,2019-T4,2020-T1,2020-T2,2020-T3,2020-T4,2021-T1,2021-T2,2021-T3,2021-T4
0,51,Alençon,0,INTERRÉGIONAL,7.2,7.5,7.3,7.5,7.7,7.4,...,8.2,8.1,7.7,7.1,8.7,7.6,7.4,7.5,7.7,7.0
1,52,Arles,0,INTERRÉGIONAL,10.8,11.0,10.7,11.1,11.2,10.8,...,11.1,10.7,10.4,9.8,11.8,10.3,10.3,10.7,10.5,9.9
2,53,Avignon,0,INTERRÉGIONAL,9.4,9.6,9.6,9.7,9.9,9.7,...,10.6,10.3,9.8,9.0,11.3,10.1,10.1,10.1,10.3,9.6
3,54,Beauvais,0,INTERRÉGIONAL,7.1,7.3,7.3,7.8,7.9,7.8,...,8.6,8.4,8.1,7.6,9.3,8.1,8.0,8.1,8.3,7.6
4,55,Bollène-Pierrelatte,0,INTERRÉGIONAL,9.5,9.5,9.4,9.4,9.6,9.5,...,10.7,10.3,9.9,9.0,11.0,9.9,9.9,10.1,10.1,9.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,9403,Calvi,94,CORSE,11.8,11.7,11.4,11.3,11.2,11.5,...,10.0,10.4,10.3,11.0,13.8,9.1,8.9,9.3,9.3,8.3
298,9404,Corte,94,CORSE,7.6,7.1,7.7,7.5,7.5,7.5,...,7.3,7.1,7.0,6.5,7.8,6.3,5.9,6.2,6.3,5.5
299,9405,Ghisonaccia,94,CORSE,10.3,9.5,9.9,10.3,10.3,10.2,...,9.0,8.8,8.4,7.3,9.6,7.7,7.4,7.7,7.9,7.1
300,9406,Porto-Vecchio,94,CORSE,9.8,9.8,9.9,10.0,10.0,10.0,...,9.1,10.3,10.1,10.8,13.6,9.1,9.1,9.1,8.5,7.7


In [32]:
chomage_df.dtypes

ZE2020         int64
LIBZE2020     object
REG            int64
LIBREG        object
2003-T1      float64
              ...   
2020-T4      float64
2021-T1      float64
2021-T2      float64
2021-T3      float64
2021-T4      float64
Length: 80, dtype: object

In [33]:
#modification du style des 2 colonnes avec code ZE pour pouvoir effectuer le MERGE
chomage_df['ZE2020'] = chomage_df['ZE2020'].astype(str)
INSEE1_df['code_ze'] = INSEE1_df['code_ze'].astype(str)

In [34]:
#MERGE : création de INSEE2_df en conctenant INSEE1_df avec les colonnes ZE2020, LIBZE2020 et 2021-T4 de chomage_df avec clé ZE2020 (numéro de ZE)
INSEE2_df = pd.merge(INSEE1_df, chomage_df, how = "left", left_on="code_ze", right_on="ZE2020")
INSEE2_df

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze,ZE2020,LIBZE2020,REG,...,2019-T3,2019-T4,2020-T1,2020-T2,2020-T3,2020-T4,2021-T1,2021-T2,2021-T3,2021-T4
0,1400,01001,L ABERGEMENT CLEMENCIAT,01001,L'Abergement-Clémenciat,8405,Bourg en Bresse,8405,Bourg en Bresse,84.0,...,6.2,6.1,5.8,5.6,6.9,6.0,6.0,5.9,6.0,5.5
1,1640,01002,L ABERGEMENT DE VAREY,01002,L'Abergement-de-Varey,8405,Bourg en Bresse,8405,Bourg en Bresse,84.0,...,6.2,6.1,5.8,5.6,6.9,6.0,6.0,5.9,6.0,5.5
2,1500,01004,AMBERIEU EN BUGEY,01004,Ambérieu-en-Bugey,8405,Bourg en Bresse,8405,Bourg en Bresse,84.0,...,6.2,6.1,5.8,5.6,6.9,6.0,6.0,5.9,6.0,5.5
3,1330,01005,AMBERIEUX EN DOMBES,01005,Ambérieux-en-Dombes,8434,Villefranche-sur-Saône,8434,Villefranche-sur-Saône,84.0,...,6.4,6.3,6.0,5.8,7.0,6.2,6.2,6.1,6.1,5.5
4,1300,01006,AMBLEON,01006,Ambléon,8404,Belley,8404,Belley,84.0,...,6.3,6.0,5.7,5.7,6.9,6.1,6.2,6.1,6.1,5.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38921,20272,2B356,ZALANA,2B356,Zalana,9405,Ghisonaccia,9405,Ghisonaccia,94.0,...,9.0,8.8,8.4,7.3,9.6,7.7,7.4,7.7,7.9,7.1
38922,20214,2B361,ZILIA,2B361,Zilia,9403,Calvi,9403,Calvi,94.0,...,10.0,10.4,10.3,11.0,13.8,9.1,8.9,9.3,9.3,8.3
38923,20272,2B364,ZUANI,2B364,Zuani,9404,Corte,9404,Corte,94.0,...,7.3,7.1,7.0,6.5,7.8,6.3,5.9,6.2,6.3,5.5
38924,20243,2B365,SAN GAVINO DI FIUMORBO,2B365,San-Gavino-di-Fiumorbo,9405,Ghisonaccia,9405,Ghisonaccia,94.0,...,9.0,8.8,8.4,7.3,9.6,7.7,7.4,7.7,7.9,7.1


In [35]:
# supprime les colonnes inutiles cad 2003-T1 (colonne 5) à 2021-T3 (colonne 79)
# NE PARCHE PAS : INSEE2_df.drop(INSEE2_df.iloc[:,5:79],1,inplace=True)

In [36]:
# ne garder que les colonnes utiles
INSEE2_df = INSEE2_df[["cp", "code_commune", "commune", "code_commune2", "commune2", "code_ze", "ze", "ZE2020", "LIBZE2020", "2021-T4"]]
INSEE2_df.rename (columns = {'2021-T4' : 'chomage_2021T4'}, inplace = True)
INSEE2_df

C:\Users\emman\anaconda3\envs\projet_desu_tre\lib\site-packages\pandas\core\frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,cp,code_commune,commune,code_commune2,commune2,code_ze,ze,ZE2020,LIBZE2020,chomage_2021T4
0,1400,01001,L ABERGEMENT CLEMENCIAT,01001,L'Abergement-Clémenciat,8405,Bourg en Bresse,8405,Bourg en Bresse,5.5
1,1640,01002,L ABERGEMENT DE VAREY,01002,L'Abergement-de-Varey,8405,Bourg en Bresse,8405,Bourg en Bresse,5.5
2,1500,01004,AMBERIEU EN BUGEY,01004,Ambérieu-en-Bugey,8405,Bourg en Bresse,8405,Bourg en Bresse,5.5
3,1330,01005,AMBERIEUX EN DOMBES,01005,Ambérieux-en-Dombes,8434,Villefranche-sur-Saône,8434,Villefranche-sur-Saône,5.5
4,1300,01006,AMBLEON,01006,Ambléon,8404,Belley,8404,Belley,5.5
...,...,...,...,...,...,...,...,...,...,...
38921,20272,2B356,ZALANA,2B356,Zalana,9405,Ghisonaccia,9405,Ghisonaccia,7.1
38922,20214,2B361,ZILIA,2B361,Zilia,9403,Calvi,9403,Calvi,8.3
38923,20272,2B364,ZUANI,2B364,Zuani,9404,Corte,9404,Corte,5.5
38924,20243,2B365,SAN GAVINO DI FIUMORBO,2B365,San-Gavino-di-Fiumorbo,9405,Ghisonaccia,9405,Ghisonaccia,7.1


In [37]:
#vérifier le nombre de valeurs manquantes (null ou NA) dans chaque colonne : return TRUE where it detects a missing value
INSEE2_df.isna().sum()

cp                 0
code_commune       0
commune            0
code_commune2      0
commune2           0
code_ze            0
ze                 0
ZE2020            48
LIBZE2020         48
chomage_2021T4    48
dtype: int64

In [42]:
# identification et visualisation des 48 lignes avec des valeurs manquantes : il s'agit des codes 97 - 132 lignes
INSEE2_df[INSEE2_df.LIBZE2020.isna()]

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze,ZE2020,LIBZE2020,chomage_2021T4
38374,97390,97301,REGINA,97301,Régina,301,Est-littoral,NaN,NaN,NaN
38375,97353,97301,REGINA,97301,Régina,301,Est-littoral,NaN,NaN,NaN
38376,97300,97302,CAYENNE,97302,Cayenne,301,Est-littoral,NaN,NaN,NaN
38377,97350,97303,IRACOUBO,97303,Iracoubo,303,Savanes,NaN,NaN,NaN
38378,97310,97304,KOUROU,97304,Kourou,303,Savanes,NaN,NaN,NaN
38379,97355,97305,MACOURIA,97305,Macouria,301,Est-littoral,NaN,NaN,NaN
38380,97318,97306,MANA,97306,Mana,302,Ouest-Guyanais,NaN,NaN,NaN
38381,97360,97306,MANA,97306,Mana,302,Ouest-Guyanais,NaN,NaN,NaN
38382,97351,97307,MATOURY,97307,Matoury,301,Est-littoral,NaN,NaN,NaN
38383,97313,97308,ST GEORGES,97308,Saint-Georges,301,Est-littoral,NaN,NaN,NaN


In [ ]:
#suppression de 132 lignes n°38374 à 38506 (correspondant aux codes communes et ZE d'outremer)
# ON NE SUPPRIME PAS - IL Y A DES DOUBLONS - A VOIR AVEC VIRGILE

In [ ]:
# EXPORT DE LA DATABASE EN EXCEL pour vérifier quelles sont ces 3401 données manquantes
INSEE2_df.to_excel('INSEE_manquants.xlsx', sheet_name='chomage', index=True)
# JE NE SAIS PAS OU EST EXPORTE CETTE DATAFRAME

In [ ]:
#importer le JDD indice de concentration d'emploi par commune : droper les lignes et garder que l'année 2018

In [ ]:
#MERGER les 2 JDD avec la base précédente